
cat sklearn-iris.yaml, prootocalversion1
----
apiVersion: "serving.kserve.io/v1beta1"
kind: "InferenceService"
metadata:
  name: "sklearn-iris"
  namespace: "user1"
  annotations:
    sidecar.istio.io/inject: "false"
spec:
  predictor:
    model:
      modelFormat:
        name: sklearn
      storageUri: "gs://kfserving-examples/models/sklearn/1.0/model"
      
----
cat sklearn-v2-iris.yaml
apiVersion: "serving.kserve.io/v1beta1"
kind: "InferenceService"
metadata:
  annotations:
    sidecar.istio.io/inject: "false"
  name: "sklearn-v2-iris"
  namespace: "user1"
spec:
  predictor:
    model:
      modelFormat:
        name: sklearn
      protocolVersion: v2
      runtime: kserve-sklearnserver
      storageUri: "gs://kfserving-examples/models/sklearn/1.0/model"


In [ ]:
#!pip install kserv==0.11.1

!pip install -r requirements.txt


scikit-learn==1.3.1
numpy==1.23.5
panda==1.5.3
kserve==0.11.1

In [1]:
from kserve import (
    constants,
    KServeClient,
    V1beta1InferenceService,
    V1beta1InferenceServiceSpec,
    V1beta1PredictorSpec,
    V1beta1SKLearnSpec,
    V1beta1ModelSpec,
    V1beta1ModelFormat,
    V1beta1TFServingSpec,
    utils
)

In [2]:
import warnings
warnings.filterwarnings("ignore")
from kubernetes import client 

In [3]:
#namespace='user1'
namespace = utils.get_default_target_namespace()
service_name="sklearn-v4-iris"
storage = 'gs://kfserving-examples/models/sklearn/1.0/model'

In [4]:
# 
default_model_spec = V1beta1InferenceServiceSpec(predictor=V1beta1PredictorSpec(sklearn=V1beta1SKLearnSpec(
    storage_uri=storage)))

In [5]:
isvc = V1beta1InferenceService(api_version=constants.KSERVE_V1BETA1,
                               kind=constants.KSERVE_KIND,
                               metadata=client.V1ObjectMeta(name=service_name, namespace=namespace, annotations={'sidecar.istio.io/inject':'false'}), 
                               spec=default_model_spec)

In [6]:
KServe = KServeClient()
KServe.create(isvc)

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'sidecar.istio.io/inject': 'false'},
  'creationTimestamp': '2024-02-27T09:23:12Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:sidecar.istio.io/inject': {}}},
     'f:spec': {'.': {},
      'f:predictor': {'.': {},
       'f:sklearn': {'.': {}, 'f:name': {}, 'f:storageUri': {}}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2024-02-27T09:23:10Z'}],
  'name': 'sklearn-v4-iris',
  'namespace': 'user1',
  'resourceVersion': '10144352',
  'uid': 'b4225904-97d5-4a6d-a29c-7c95b13e41db'},
 'spec': {'predictor': {'model': {'modelFormat': {'name': 'sklearn'},
    'name': '',
    'resources': {},
    'storageUri': 'gs://kfserving-examples/models/sklearn/1.0/model'}}}}

In [7]:
KServe.get(service_name, namespace=namespace, watch=True, timeout_seconds=120)

NAME             READY      PREV    LATEST  URL
sklearn-v4-iris  Unknown       0       100
NAME             READY      PREV    LATEST  URL
sklearn-v4-iris  Unknown       0       100
NAME             READY      PREV    LATEST  URL
sklearn-v4-iris  Unknown       0       100
NAME             READY      PREV    LATEST  URL
sklearn-v4-iris  Unknown       0       100
NAME             READY      PREV    LATEST  URL
sklearn-v4-iris  True          0       100  https://sklearn-v4-iris-predictor.user1.ezua.ksi.local


In [8]:
isvc_resp=KServe.get(service_name, namespace=namespace)

In [9]:
isvc_resp

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'sidecar.istio.io/inject': 'false'},
  'creationTimestamp': '2024-02-27T09:23:12Z',
  'finalizers': ['inferenceservice.finalizers'],
  'generation': 1,
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:sidecar.istio.io/inject': {}}},
     'f:spec': {'.': {},
      'f:predictor': {'.': {},
       'f:sklearn': {'.': {}, 'f:name': {}, 'f:storageUri': {}}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2024-02-27T09:23:10Z'},
   {'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:finalizers': {'.': {},
       'v:"inferenceservice.finalizers"': {}}}},
    'manager': 'manager',
    'operation': 'Update',
    'time': '2024-02-27T09:23:12Z'},
   {'apiVersion': 'serving.kserve.io/v1beta1',
    '

In [10]:
sklearn_iris_input={
    "instances": [
      [6.8,  2.8,  4.8,  1.4],
      [6.0,  3.4,  4.5,  1.6]
    ]
}

In [11]:
import requests

In [19]:
ver_info = '/v1/models/'
operaton = ':predict'
isvc_url = isvc_resp['status']['address']['url']
rsvc_url = isvc_url + ver_info + service_name + operaton

http://sklearn-v4-iris-predictor.user1.svc.cluster.local
http://sklearn-v4-iris-predictor.user1.svc.cluster.local/v1/models/sklearn-v4-iris:predict


In [20]:
response = requests.post(rsvc_url, json=sklearn_iris_input)

In [21]:
print(response.status_code)

200


In [22]:
print(response.text)

{"predictions":[1,1]}
